In [52]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [53]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows_per_player, find_lines_with_player, del_patterns, map_emoji_to_description, remove_accents, name_wordgroups
import emoji
from googletrans import Translator


In [54]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

# First Transformation (with Pattern deletion)
The first transformation focus on the deletion of Patterns to clean the corpus.

### Filter out German data

In [55]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]

### Remove similiar rows

In [56]:
# Remove the similiar rows (The Function is imported from utils on top)
df_ger = remove_similar_rows_per_player(df_ger, df_ger['player'].unique())

### Tranform Hincapie

In [57]:
# transform Hincapié to Hincapie
df_ger.loc[df_ger['player'] == 'Piero Hincapié', 'player'] = 'Piero Hincapie'

### Transform Data to lower case

In [58]:
# transform data into lower case
data_lower = df_ger.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()



### Delete Patterns

In [59]:
# delete content patterns
data_wo_pattern = data_lower.copy()
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"{'content':", "", str(x)))

In [60]:
# define patterns to delete
pattern = ['nutze kicker', 
           'mit dem', 
           'nur €2,49', 
           'https://www.faz.net/',
           'bereits pur-abonnent?', 
           'alle antworten', 
           'hinweis zur verarbeitung', 
           'werbung','olympia-verlags', 
           'bild', 'g+j medien gmbh', 
           'services',
           'kurz-link dieses artikels lautet:', 
           'http://epaper.welt.de',
           'stephan von nocks',
           'warum sehe ich faz.net',
           'permalink:',
           'mcfit mitgliedschaft',
           'fitx-vertrag',
           'kündigeneasyfitness',
           'proteinbedarf',
           'fitnessland',
           'kündigeneasyfitness',
           'trainingspuls berechnen',
           'pulsrechner',
           'fitseveneleven-mitgliedschaft',
           'alkoholabbau & promille',
           'index.promillerechner',
           'mitgliedschaft kündigen',
           'promillerechner',
           'ihr body-mass-index',
           'bmi rechner',
           'kalorienrechner',
           'grundumsatz & kalorienbedarf',
           'partnerangebote',
           'newsletter',
           'journalismus der presse',
           'abonnieren',
           '(apa)',
           'foto:',
           'quelle:',
           'lesezeit:',
           'lesen sie mehr',
           'stellenmarkt der sz.',
           'bewerben sie sich jetzt',
           'gutscheine',
           'vergleichsportal',
           'stern plus-inhalten',
           'jederzeit kündbar',
           'bereits registriert?',
           'zur startseite',
           'öffnet in neuem tab oder fenster',
           'vollansicht der tabelle unter',
           'frankfurter allgemeine zeitung'      
           ]

In [61]:
# delete patterns
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns(str(x), pattern))

### Transform Emojis to text

In [62]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'de',), str(x)))

### Remove noise

In [63]:
data_rm_1 = data_wo_pattern.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_1['data'] = data_rm_1['data'].apply(strip_numeric)

#strip links
data_rm_1['data'] = data_rm_1['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_1['data'] = data_rm_1['data'].apply(strip_multiple_whitespaces)

#strip spanish accents
data_rm_1['data'] = data_rm_1['data'].apply(lambda x: remove_accents(str(x)))

### Reset index   

In [64]:
# remove index 
data_rm_1 = data_rm_1.reset_index(drop=True)

### Remove empty rows

In [65]:
# remove empty rows 
data_rm_1 = data_rm_1.replace('', pd.NA)
data_rm_1.dropna(inplace=True)

## Store first Transformation Data 

In [66]:
# store data 
data_rm_1.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_1.csv', index=False)

# Second transformation Preperation for sentiment

In [67]:
# copy data 
data_rm_2 = data_rm_1.copy()

### Remove stopwords

In [68]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts', 'kein', 'keine', 'keinem', 'keinen', 'keiner', 'keines', 'anders']

stop_words_to_add = ['vor', 'in', 'den', 'dem', 'beim', 'wir', 'der', 'ist','ende', 'seite', 'ersten', 'fürs', 'eh', 'blick', 'schon', 'zumal', 'erst', 'mal', 'bild', 't', '(dpa)', 'fur']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

for word in stop_words_to_add:
  german_stop_words.append(word)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm_2['data'] = data_rm_2['data'].apply(lambda x: remove_stopwords(x, german_stop_words))


### Strip punctuation and remove short words
copy of unstriped data is taken because it's needed for further data preprocessing 

In [69]:
# copy data for third transformation
data_rm_3 = data_rm_2.copy()

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_2['data'] = data_rm_2['data'].apply(strip_punctuation)

#strip_short deletes words smaller 3
data_rm_2['data'] = data_rm_2['data'].apply(strip_short)

## Store second transformation data

In [70]:
data_rm_2.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_2.csv', index=False)

# Third Transformation (Paragraphs with Playernames)
The third transformation focus on the deletion of sentences to clean the corpus. The only paragraphs kept are the one including the player names.

### Get lines and following lines where the Player name appears in the corpus 

In [71]:
# select only paragraphs which include playernames 
data_with_playernames = find_lines_with_player(data_rm_3, data_rm_3['player'].unique(), n_lines = 0)

### Create wordgroups
Remove first names of players and create word pairs e.g. for bayer leverkusen -> bayerleverkusen or europa league -> europaleague

In [72]:
# perform wordpair function
data_with_playernames = name_wordgroups(data_with_playernames)

### Delete playernames from their sentences
Because the playernames took a huge influence on the clustering they will be removed for each player. 

In [73]:
# for every player remove their names from the texts 
for player in data_with_playernames['player'].unique():
    f_l_name = player.split()

    # Extracting the first name
    first_name = str(f_l_name[0])

    # Extracting the last name
    last_name = str(f_l_name[1])

    updated_pattern = r"\b(" + first_name.lower() + r"|" + last_name.lower() + r")\b|"


    # Apply the function to the data column
    data_with_playernames.loc[data_with_playernames['player'] == player, 'data'] = data_with_playernames.loc[data_with_playernames['player'] == player, 'data'].apply(lambda x: re.sub(updated_pattern, "", str(x)))



### Strip punctuation and remove short words

In [74]:
# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_with_playernames['data'] = data_with_playernames['data'].apply(strip_punctuation)

#strip_short deletes words smaller 3
data_with_playernames['data'] = data_with_playernames['data'].apply(strip_short)

### Delete empty rows and store csv

In [75]:
# delet empty rows
data_with_playernames = data_with_playernames.replace('', pd.NA)
data_with_playernames.dropna(inplace=True)

## Store third transformation data

In [76]:
data_with_playernames.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_condensed.csv', index=False)